In [1]:
import torch
import numpy as np


def job(factor):
    masks = torch.load(f'/nvme2/yujiepan/workspace/moe-bert/moe-bert-learning/Transkimmer/model/glue/0613-eval-ft-start/factor{factor}/skim_masks.pth', map_location='cpu')
    with open(f'/nvme2/yujiepan/workspace/moe-bert/moe-bert-learning/Transkimmer/model/glue/0613-eval-ft-start/factor{factor}/stderr.log', 'r') as f:
        content = f.read()
        acc = content.split("eval: {'accuracy': ")[-1].split('}')[0]
        # print(acc)
    num_samples = len(masks[0])
    # print(num_samples)
    from collections import defaultdict
    L1 = defaultdict(list)
    L2 = defaultdict(list)

    for i in range(num_samples):
        for layer_id in range(12):
            l = masks[layer_id][i].float().sum().item()
            L1[i].append(l)
            L2[i].append(l**2)

    L1 = np.array(list(L1.values()))
    L2 = np.array(list(L2.values()))
    return dict(a1=50.973092, a2=2947.33845, l1=L1.mean(), l2=L2.mean(), save1=100 * (L1.mean() / 50.973092 - 1), save2=100 * (L2.mean() / 2947.33845 - 1), factor=factor, acc=acc)


In [2]:
results = []
# for factor in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
# for factor in [0.2, 0.15] + [0.25, 0.05,0.1,0.3] + [0.21, 0.22, 0.23, 0.24]:
for factor in [0.35, 0.4, 0.45, 0.5]:
    results.append(job(factor))

import pandas as pd
pd.DataFrame(results).sort_values('save2')


,a1,a2,l1,l2,save1,save2,factor,acc
3,50.973092,2947.33845,26.614604,957.409497,-47.786953,-67.516133,0.50,0.8982244188174996
2,50.973092,2947.33845,28.549134,1068.309766,-43.991756,-63.753407,0.45,0.9009701629141498
1,50.973092,2947.33845,30.472756,1175.224510,-40.217956,-60.125906,0.40,0.9044481054365733
0,50.973092,2947.33845,33.794908,1386.560300,-33.700494,-52.955511,0.35,0.90591250228812
